<a href="https://colab.research.google.com/github/aniparnagian/Artsakh_War_Sentiment-and_LDA_Project-/blob/main/Gensim_Tweets_Pashinyan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LDA Topic Modeling with Gensim

Heavily modified and expanded based upon Gensim tutorial

By Jon Chun
28 Mar 2022 Updated

# Setup and Configuration

In [1]:
!pip install python-slugify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 19.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=35d6211b8defb88f2dcb704d4ea83f71b9a043f11876b812b7610a825b7b7920
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=96722d33f171a22411705d884dfbace7b76ee4a05e5b961b85da60ac2a791927
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyldavis sklearn


In [3]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 36.3 MB/s 
     |████████████████████████████████| 110 kB 46.4 MB/s 


## Configure Jupyter Notebook

In [4]:
## Configure Jupyter Notebook

# Ignore warnings

import warnings
warnings.filterwarnings('ignore')

# Enable multiple outputs from one code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
from IPython.display import Image
from ipywidgets import widgets, interactive

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Import Libraries

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [6]:
from slugify import slugify

# Get Text Data


## Upload Raw Textfile

In [7]:
from google.colab import files

TEXT_ENCODING = 'utf-8'

In [8]:
%%time

# NOTE: 1m07s

# Upload Plain Text File
uploaded = files.upload()

# NOTE: Allows for multiple file uploads, will only process the last
#       Left in for future feature addition (processing multiple files at once)
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  text_filename_str = fn

# Extract from Dict and decode binary into char string
text_raw_str = uploaded[text_filename_str].decode(TEXT_ENCODING)

Saving tweets_raw_pashinyan_20211223_145418_20221214_234645.csv to tweets_raw_pashinyan_20211223_145418_20221214_234645.csv
User uploaded file "tweets_raw_pashinyan_20211223_145418_20221214_234645.csv" with length 8349220 bytes
CPU times: user 922 ms, sys: 162 ms, total: 1.08 s
Wall time: 56.3 s


In [9]:
# Recover parameters of original search from filename components
word1, word2, search_str, start_date, start_time, end_date, end_time = text_filename_str.split('_')
end_time = ''.join(end_time.split('.')[:-1])
end_time = ''.join(end_time.split('(')[0])

print(f'word1: {word1}')
print(f'word2: {word2}')
print(f'search_str: {search_str}')
print(f'start_date: {start_date}')
print(f'start_time: {start_time}')
print(f'end_date: {end_date}')
print(f'end_time: {end_time}')

word1: tweets
word2: raw
search_str: pashinyan
start_date: 20211223
start_time: 145418
end_date: 20221214
end_time: 234645


In [10]:
# Read raw tweet datafile into DataFrame 'data'

try:
  headers = ['datetime','id','tweet','username','reply_ct','retweet_ct','like_ct','quote_ct','retweeted_id','quoted_id',
             'text_raw','text_noemoji','text_clean','vader','textblob','distilbert'] # ,'roberta']
  dtypes = {'datetime':'str',
            'id':'int',
            'tweet':'string',
            'username':'string',
            'reply_ct':'int',
            'retweet_ct':'int',
            'like_ct':'int',
            'quote_ct':'int',
            'retweeted_id':'string',
            'quoted_id':'string',
            'text_raw':'string',
            'text_noemoji':'string',
            'text_clean':'string',
            'vader':'float',
            'textblob':'float',
            'distilbert':'float'}
            # 'roberta':'float'}
  parse_dates = ['datetime']
  # text_df = pd.read_csv(text_filename_str, skiprows=[0], names=headers, dtype=dtypes, parse_dates=parse_dates, index_col=None)
  text_df = pd.read_csv(text_filename_str, index_col=None)
  text_df.head()
  text_df.info()
except:
  print(f'ERROR: Tweets datafile cannot be read by pd.read_csv')

,datetime,id,tweet,username,reply_ct,retweet_ct,like_ct,quote_ct,retweeted_id,quoted_id
0,2021-12-23 14:54:18+00:00,1474030612453470215,"#armenianbreakingnews\nTomorrow, 20:00, Pashin...",Armbreakingnews,1,3,5,1,NaN,NaN
1,2021-12-23 15:09:53+00:00,1474034534605860864,What “trap” did Pashinyan set for the oppositi...,gagrulenet,0,0,0,0,NaN,NaN
2,2021-12-23 15:13:55+00:00,1474035546557239303,"#armenianbreakingnews\n1/2\nOpposition blocs ""...",Armbreakingnews,1,0,1,0,NaN,NaN
3,2021-12-23 15:14:09+00:00,1474035607966044164,2/2\nThe format of a closed meeting on issues ...,Armbreakingnews,0,0,1,0,NaN,NaN
4,2021-12-23 15:36:16+00:00,1474041173295747080,Another victory day. \nPashinyan’s “government...,shushatrash,0,0,2,0,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   datetime      30000 non-null  object 
 1   id            30000 non-null  int64  
 2   tweet         30000 non-null  object 
 3   username      30000 non-null  object 
 4   reply_ct      30000 non-null  int64  
 5   retweet_ct    30000 non-null  int64  
 6   like_ct       30000 non-null  int64  
 7   quote_ct      30000 non-null  int64  
 8   retweeted_id  0 non-null      float64
 9   quoted_id     2899 non-null   object 
dtypes: float64(1), int64(5), object(4)
memory usage: 2.3+ MB


## Clean DataFrame

In [11]:
# Drop unused columns

if 'reply_ct' in text_df.columns:
  text_df.drop(columns=['reply_ct'], inplace=True)

if 'retweet_ct' in text_df.columns:
  text_df.drop(columns=['retweet_ct'], inplace=True)

if 'like_ct' in text_df.columns:
  text_df.drop(columns=['like_ct'], inplace=True)

if 'quote_ct' in text_df.columns:
  text_df.drop(columns=['quote_ct'], inplace=True)

if 'retweeted_id' in text_df.columns:
  text_df.drop(columns=['retweeted_id'], inplace=True)

if 'quoted_id' in text_df.columns:
  text_df.drop(columns=['quoted_id'], inplace=True)


In [12]:
# Clean and drop illegal datetime values

text_df['datetime'] = pd.to_datetime(text_df['datetime'], errors='coerce')
text_df = text_df.dropna(subset=['datetime'])
text_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   datetime  30000 non-null  datetime64[ns, UTC]
 1   id        30000 non-null  int64              
 2   tweet     30000 non-null  object             
 3   username  30000 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 1.1+ MB


In [13]:
# Drop any row that have at any NaN

text_df.dropna(inplace=True)
text_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   datetime  30000 non-null  datetime64[ns, UTC]
 1   id        30000 non-null  int64              
 2   tweet     30000 non-null  object             
 3   username  30000 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 1.1+ MB


In [14]:
text_df['text_raw'] = text_df['tweet']

text_df['text_raw_type'] = text_df['text_raw'].apply(lambda x: type(x))
text_df['text_raw_type'].value_counts()

<class 'str'>    30000
Name: text_raw_type, dtype: int64

In [15]:
text_df.convert_dtypes()
text_df.info()

,datetime,id,tweet,username,text_raw,text_raw_type
0,2021-12-23 14:54:18+00:00,1474030612453470215,"#armenianbreakingnews Tomorrow, 20:00, Pashiny...",Armbreakingnews,"#armenianbreakingnews Tomorrow, 20:00, Pashiny...",<class 'str'>
1,2021-12-23 15:09:53+00:00,1474034534605860864,What “trap” did Pashinyan set for the oppositi...,gagrulenet,What “trap” did Pashinyan set for the oppositi...,<class 'str'>
2,2021-12-23 15:13:55+00:00,1474035546557239303,"#armenianbreakingnews 1/2 Opposition blocs ""Ar...",Armbreakingnews,"#armenianbreakingnews 1/2 Opposition blocs ""Ar...",<class 'str'>
3,2021-12-23 15:14:09+00:00,1474035607966044164,2/2 The format of a closed meeting on issues r...,Armbreakingnews,2/2 The format of a closed meeting on issues r...,<class 'str'>
4,2021-12-23 15:36:16+00:00,1474041173295747080,Another victory day. Pashinyan’s “government”...,shushatrash,Another victory day. Pashinyan’s “government”...,<class 'str'>
...,...,...,...,...,...,...
29995,2022-12-14 22:01:30+00:00,1603148198511525889,"Pashinyan's wife posting this while 120,000 Ar...",451_AD,"Pashinyan's wife posting this while 120,000 Ar...",<class 'str'>
29996,2022-12-14 23:00:01+00:00,1603162927086718977,Accounts that never criticize the Pashinyan re...,TheAntiNikol,Accounts that never criticize the Pashinyan re...,<class 'str'>
29997,2022-12-14 23:18:45+00:00,1603167640784367616,"@MateoDeZurda @JotaceGr Bueno, Pashinyan no es...",mundoarmenio,"@MateoDeZurda @JotaceGr Bueno, Pashinyan no es...",<class 'str'>
29998,2022-12-14 23:41:07+00:00,1603173269393141760,"Now to #Armenia proper. They aren't just the ""...",Proud2BeHate,"Now to #Armenia proper. They aren't just the ""...",<class 'str'>


<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   datetime       30000 non-null  datetime64[ns, UTC]
 1   id             30000 non-null  int64              
 2   tweet          30000 non-null  object             
 3   username       30000 non-null  object             
 4   text_raw       30000 non-null  object             
 5   text_raw_type  30000 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(4)
memory usage: 1.6+ MB


In [16]:
# Try to convert Tweet text to string (raise error if incompatible encoding)

# text_df['text_raw'] = text_df['text_raw'].apply(lambda x: str(x)) # 
# text_df[['text_raw','username','tweet','text_noemoji','text_clean']] = text_df[['text_raw','username','tweet','text_noemoji','text_clean']].astype('string', errors='raise') # , errors='coerce')
text_df[['text_raw','username','tweet']] = text_df[['text_raw','username','tweet']].astype('string', errors='raise') # , errors='coerce')
# text_df = text_df.dropna(subset=['text_raw'])
text_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   datetime       30000 non-null  datetime64[ns, UTC]
 1   id             30000 non-null  int64              
 2   tweet          30000 non-null  string             
 3   username       30000 non-null  string             
 4   text_raw       30000 non-null  string             
 5   text_raw_type  30000 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(1), string(3)
memory usage: 1.6+ MB


In [18]:
line_ls = text_df.text_raw.to_list()
print(f'BEFORE filtering out NAType: len(line_ls): {len(line_ls)}')

# line_ls = [x for x in line_ls if isinstance(x, str)]
# print(f' AFTER filtering out NAType: len(line_ls): {len(line_ls)}')

# line_ls = [x for x in line_ls if len(x) > 0]
# print(f' AFTER filtering out null lines: len(line_ls): {len(line_ls)}')

BEFORE filtering out NAType: len(line_ls): 30000


In [19]:
# Agglomerate n Tweets into a Synthetic Document for Topic Modeling

doc_ls = []
tweets_per_doc = 10
doc_current_ls = []
j = 0

for i, aline in enumerate(line_ls):
  
  if (((i % tweets_per_doc) == 0) & (i>0)):
    doc_temp_str = '\n'.join(doc_current_ls)
    doc_ls.append(doc_temp_str)
    print(f'Synthetic Doc #{j}: {doc_temp_str}\n\n')
    j += 1
    # Reset current syn doc
    doc_current_ls = []
    doc_current_ls.append(aline)

  else:
    try:
      doc_current_ls.append(aline)
    except:
      print(f'ERROR: Cannot strip()')

  # line_doc_ls.append(aline)

print('\n\n')

text_raw_str = "\n\n".join(doc_ls)
text_raw_str[:5]


Streaming output truncated to the last 5000 lines.
@hyepatriarch I’m curious, If one looks at our history, appeasement has not worked well in our favor either. Soviet Union, Turkey, Persia, Assyria, etc… 

History will judge but Pashinyan is not doing anything new. It’s tried and failed - no diff than Koch/Sarg. Two flips of the same coin.
Día 25 de protestas en Armenia: “Nikol Pashinyan no tiene mandato para negociar en nombre del pueblo armenio” https://t.co/ha7uXRMaQO https://t.co/H01lDhBMv9
@NikolPashinyan @MinPres He is attacking a nation just got out of very cruel war caused by pashinyan himself. He is arresting woman and children. https://t.co/sUoNgCJ8M8


Synthetic Doc #937: @FizuliJ @AlexGunnar3 @AvetissianAn But that’s the problem. Azeri’s ARE IN Armenia. It’s simply that pashinyan isnt doing anything and wants to leave everything behind and start from 0, forgetting all the struggles, deaths, etc.
Children have rights, including #Armenian 🇦🇲 children &amp; should not be dragg

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



'#arme'

In [20]:
doc_ls[-1]
len(doc_ls)

'@Gegham_Artsakh If Pashinyan is smart, he\'ll see that the separatist-terrorists can\'t feed themselves and call them to Irevan, much cheaper for him.\nThen he can sign a peace  agreement.\nThe last time Aliyev, Pashinyan, and Putin met in a trilateral format was on 31 October in Sochi. In recent weeks, especially after Prague, Moscow has become particularly re-engaged with the sides while some analysts wonder if the EU’s Brussels process can still continue. 2/2\n@comradepriya nikol pashinyan rolls worst Artsakh defense ever, asked to take estradiol\n@Artak_Beglaryan @UNArmenia @UN Pashinyan speech: Karabakh is Azerbaijan\nhttps://t.co/bFCyOYnOYm\n@TatevikHayrape1 What is pashinyan doing about this? he is with the enemy, he has to mobilize the army and do joint military exercise with Iran, so shameful for our nation that others are willing to do more for our people then our own government\nThe peaceful protestors ended their protest in Republic Square by lighting candles in support of

2999

# Enter Title

In [21]:
# If this is a Twitter textfile, the search terms are recovered from the filename

# Recover parameters of original search from filename components
try:
  word1, word2, search_str, start_date, start_time, end_date, end_time = text_filename_str.split('_')

  print(f'word1: {word1}')
  print(f'word2: {word2}')
  print(f'search_str: {search_str}')
  print(f'start_date: {start_date}')
  print(f'end_date: {end_date}')
  print('\n')
  
  text_title_str = f'[{search_str}] from [{start_date}] to [{end_date}]'
  print(f'\nTwitter Search String:\n\n  {text_title_str}\n')

except:
  print('This is not a Twitter datafile named in standard format\n\nEnter the Novel information below.')

word1: tweets
word2: raw
search_str: pashinyan
start_date: 20211223
end_date: 20221214



Twitter Search String:

  [pashinyan] from [20211223] to [20221214]



In [22]:
#@title Enter Title "[Title] by [Author]""

Text_Title = "[pashinyan] from [20211223] to [20221214]" #@param {type:"string"}

# Remove illegal filename punctuation
Text_Title = Text_Title.replace("'","")
Text_Title = Text_Title.replace('"','')
Text_Title = Text_Title.replace('.',' ')

text_title_str = ' '.join([x.capitalize() for x in slugify(Text_Title).replace('-',' ').split()])
text_title_str

'Pashinyan From 20211223 To 20221214'


# LDA Model

Introduces Gensim's LDA model and demonstrates its use on the NIPS corpus.


The purpose of this tutorial is to demonstrate how to train and tune an LDA model.

In this tutorial we will:

* Load input data.
* Pre-process that data.
* Transform documents into bag-of-words vectors.
* Train an LDA model.

This tutorial will **not**:

* Explain how Latent Dirichlet Allocation works
* Explain how the LDA model performs inference
* Teach you all the parameters and options for Gensim's LDA implementation

If you are not familiar with the LDA model or how to use it in Gensim, I (Olavur Mortensen)
suggest you read up on that before continuing with this tutorial. Basic
understanding of the LDA model should suffice. Examples:

* `Introduction to Latent Dirichlet Allocation <http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation>`_
* Gensim tutorial: `sphx_glr_auto_examples_core_run_topics_and_transformations.py`
* Gensim's LDA model API docs: :py:class:`gensim.models.LdaModel`

I would also encourage you to consider each step when applying the model to
your data, instead of just blindly applying my solution. The different steps
will depend on your data and possibly your goal with the model.





## Segment Text

In [23]:
text_raw_str[:500]
print('\n')
len(text_raw_str)

'#armenianbreakingnews\nTomorrow, 20:00, Pashinyan will answer questions from #media/#NGO on #Facebook. The question will be sent earlier. He will have the option to choose questions/prepare his answers. Last time the comments were turned off because of the negative replies. https://t.co/MGqQLoG2yC\nWhat “trap” did Pashinyan set for the opposition? fool me once shame on you fool me twice shame on\xa0me https://t.co/giEBbu57E4\n#armenianbreakingnews\n1/2\nOpposition blocs "Armenia" and "With Honor" made a'

5943626

In [24]:
"""
# Split into paragraphs
text_parags_ls = text_raw_str.split('\n')
print(f'There are {len(text_parags_ls)} raw paragraphs\n')

# Delete empty paragraphs
text_parags_ls = [x for x in text_parags_ls if len(x.strip()) > 0]
print(f'There are {len(text_parags_ls)} non-empty paragraphs\n\n')

# Replace stray/embedded /n with a space
text_parags_clean_ls = []
for aparag in text_parags_ls:
  text_parags_clean_ls.append(aparag.replace('\n',' ').strip())

text_parags_ls = text_parags_clean_ls
text_parags_ls[:5] 
"""

text_parags_ls = doc_ls

"\n# Split into paragraphs\ntext_parags_ls = text_raw_str.split('\n')\nprint(f'There are {len(text_parags_ls)} raw paragraphs\n')\n\n# Delete empty paragraphs\ntext_parags_ls = [x for x in text_parags_ls if len(x.strip()) > 0]\nprint(f'There are {len(text_parags_ls)} non-empty paragraphs\n\n')\n\n# Replace stray/embedded /n with a space\ntext_parags_clean_ls = []\nfor aparag in text_parags_ls:\n  text_parags_clean_ls.append(aparag.replace('\n',' ').strip())\n\ntext_parags_ls = text_parags_clean_ls\ntext_parags_ls[:5] \n"

In [25]:
# Verify first paragraph (after title and chapter headings)

text_parags_ls[3]
print('\n')
# Paragraph char count
len(text_parags_ls[3])

"@GeneralLmao @geno1915 @menaaffairscom @ATuranQ @ANCA_DC That is more Pashinyan's problem then  Turkeys.\nNikol Pashinyan préside la réunion du Conseil de sécurité https://t.co/TUi1uF62fX https://t.co/VGLkVqxyT9\nMoscou décrit la réunion de Pashinyan et Aliyev sous la médiation de l'UE comme «\xa0le développement d'accords trilatéraux\xa0» de Sotchi https://t.co/d9azAHK9IH https://t.co/jtGbf99tIt\n#Armenia #Turkey #Iran #peace #Pashinyan #Ter_Petrosyan\nhttps://t.co/fwCWBUFomh\n#Armenia 2025 bond fall 110, 15% yield, tentative post-war thaw #Turkey complicated currency swap consider #us dtry save #Azerbaijan https://t.co/Utlk4zryug.  PM Pashinyan re-elect relative #Russia distance, Baku-Ankara energy pact peril. genocide history hump. #EmergingMarkets\n@301_AD Armenia needs sb who isnt pashinyan, kocharyan, ter-petrosyan etc. A law that shortens maximum amount of terms of PMs to 2 times maximum per politician no matter what would be great\nTurkey thinks Armenian reelecting Turkish Age

1657

In [26]:
# Delete any paragraphs shorter than MIN_LEN_PARAG

MIN_LEN_PARAG = 5
MIN_LEN_DOC = 100

# Delete any paragraphs shorter than MIN_LEN_PARAG
text_parags_ls = [x for x in text_parags_ls if len(x) > MIN_LEN_PARAG]

# Trim any leading/trailing/multiple embedded whitespaces
text_parags_ls = [' '.join(x.split()) for x in text_parags_ls]

len(text_parags_ls)

2999

In [27]:
# Agglomerate paragraphs into Documents of MIN_LEN_DOC=1000 chars
"""
parag_ct = len(text_parags_ls)

doc_now_str = ''
doc_now_len = 0
docs = []

for i in range(parag_ct):
  # print(f'Processing Paragraph #{i}')
  parag_now_str = text_parags_ls[i]
  doc_now_str += parag_now_str
  doc_now_len += len(parag_now_str)
  if doc_now_len > MIN_LEN_DOC:
    docs.append(doc_now_str)
    doc_now_str = ''
    doc_now_len = 0

docs[-1] += doc_now_str

print(f'There are now {len(docs)} Documents of {MIN_LEN_DOC} chars or more')
"""

"\nparag_ct = len(text_parags_ls)\n\ndoc_now_str = ''\ndoc_now_len = 0\ndocs = []\n\nfor i in range(parag_ct):\n  # print(f'Processing Paragraph #{i}')\n  parag_now_str = text_parags_ls[i]\n  doc_now_str += parag_now_str\n  doc_now_len += len(parag_now_str)\n  if doc_now_len > MIN_LEN_DOC:\n    docs.append(doc_now_str)\n    doc_now_str = ''\n    doc_now_len = 0\n\ndocs[-1] += doc_now_str\n\nprint(f'There are now {len(docs)} Documents of {MIN_LEN_DOC} chars or more')\n"

In [28]:
# View the first 5 docs

doc_ls[:5]

['#armenianbreakingnews\nTomorrow, 20:00, Pashinyan will answer questions from #media/#NGO on #Facebook. The question will be sent earlier. He will have the option to choose questions/prepare his answers. Last time the comments were turned off because of the negative replies. https://t.co/MGqQLoG2yC\nWhat “trap” did Pashinyan set for the opposition? fool me once shame on you fool me twice shame on\xa0me https://t.co/giEBbu57E4\n#armenianbreakingnews\n1/2\nOpposition blocs "Armenia" and "With Honor" made a joint statement where they informed that Alen Simonyan has conveyed a proposal to the opposition to hold a closed meeting with #Pashinyan. https://t.co/iZP3l1Gibl\n2/2\nThe format of a closed meeting on issues related to the #Armenian people is unacceptable, they will only take part in meetings openly.\nIn October 2020, during the #Artsakh war, closed meetings with #Pashinyan took place, to this day, the outcome is distorted by #Pashinyan\nAnother victory day. \nPashinyan’s “governmen

In [29]:
# View the first 500 chars in the 50th doc

doc_ls[49][:500]

'#armenianbreakingnews\n1/8\nTurkish https://t.co/6yGwBZONgH published an article where It was revealed that Pashinyan was working with Azerbaijani intelligence\nhttps://t.co/6J8ayHVc9C\n2/8\nA former employee of the #Azerbaijani intelligence, in exile, claimed that #Armenian PM was introduced to Azerbaijani special service in 2008. Later, efforts were made so Pashinyan becomes the leader of the Armenian opposition, and all kinds of financial support were granted.\n3/8\nThe former employee said that a s'

In [30]:
# View the last 100 chars in the last doc

doc_ls[-1][-100:]

've brought Armenians to this place. . .so Armenians must remove Nikol and Civil Contract from power.'

## Pre-process and vectorize the documents

As part of preprocessing, we will:

* Expand Contractions
* Tokenize (split the documents into tokens)
* Define stopwords and filter out
* Lemmatize the tokens.
* Compute bigrams.
* Compute a bag-of-words representation of the data.

First we tokenize the text using a regular expression tokenizer from NLTK. We
remove numeric tokens and tokens that are only a single character, as they
don't tend to be useful, and the dataset contains a lot of them.

.. Important::

   This tutorial uses the nltk library for preprocessing, although you can
   replace it with something else if you want.




### Steps

#### Expand Contractions

In [31]:
# Expand Contractions (e.g. can't -> can not)

import contractions

# Test
contractions.fix("yall're happy now it isn't a don't tell problem", slang=False) # default: true

"yall're happy now it is not a do not tell problem"

In [33]:
doc_ls[:5]

['#armenianbreakingnews\nTomorrow, 20:00, Pashinyan will answer questions from #media/#NGO on #Facebook. The question will be sent earlier. He will have the option to choose questions/prepare his answers. Last time the comments were turned off because of the negative replies. https://t.co/MGqQLoG2yC\nWhat “trap” did Pashinyan set for the opposition? fool me once shame on you fool me twice shame on\xa0me https://t.co/giEBbu57E4\n#armenianbreakingnews\n1/2\nOpposition blocs "Armenia" and "With Honor" made a joint statement where they informed that Alen Simonyan has conveyed a proposal to the opposition to hold a closed meeting with #Pashinyan. https://t.co/iZP3l1Gibl\n2/2\nThe format of a closed meeting on issues related to the #Armenian people is unacceptable, they will only take part in meetings openly.\nIn October 2020, during the #Artsakh war, closed meetings with #Pashinyan took place, to this day, the outcome is distorted by #Pashinyan\nAnother victory day. \nPashinyan’s “governmen

In [34]:
len(doc_ls)

2999

In [35]:
# Expand all Contractions paragraph by paragraph

"""
docs_clean_ls = []
for adoc in doc_ls:
  docs_clean_ls.append(contractions.fix(adoc))

docs = docs_clean_ls
docs[:5]
""";

docs = doc_ls

'\ndocs_clean_ls = []\nfor adoc in doc_ls:\n  docs_clean_ls.append(contractions.fix(adoc))\n\ndocs = docs_clean_ls\ndocs[:5]\n'

#### Tokenize Text

In [36]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [37]:
# Verify docs

# content and token count of first doc
docs[0]
print(f'\n\nThere are {len(docs[0])} tokens in the first document')

['armenianbreakingnews',
 'tomorrow',
 'pashinyan',
 'will',
 'answer',
 'questions',
 'from',
 'media',
 'ngo',
 'on',
 'facebook',
 'the',
 'question',
 'will',
 'be',
 'sent',
 'earlier',
 'he',
 'will',
 'have',
 'the',
 'option',
 'to',
 'choose',
 'questions',
 'prepare',
 'his',
 'answers',
 'last',
 'time',
 'the',
 'comments',
 'were',
 'turned',
 'off',
 'because',
 'of',
 'the',
 'negative',
 'replies',
 'https',
 'co',
 'mgqqlog2yc',
 'what',
 'trap',
 'did',
 'pashinyan',
 'set',
 'for',
 'the',
 'opposition',
 'fool',
 'me',
 'once',
 'shame',
 'on',
 'you',
 'fool',
 'me',
 'twice',
 'shame',
 'on',
 'me',
 'https',
 'co',
 'giebbu57e4',
 'armenianbreakingnews',
 'opposition',
 'blocs',
 'armenia',
 'and',
 'with',
 'honor',
 'made',
 'joint',
 'statement',
 'where',
 'they',
 'informed',
 'that',
 'alen',
 'simonyan',
 'has',
 'conveyed',
 'proposal',
 'to',
 'the',
 'opposition',
 'to',
 'hold',
 'closed',
 'meeting',
 'with',
 'pashinyan',
 'https',
 'co',
 'izp3l1gib



There are 336 tokens in the first document


#### Customize Stopwords

In [38]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stopwords_ls = stopwords.words('english')

print(f'\nThe first ten stopwords:')
stopwords_ls[20:]
print(f'\n\nThere are [{len(stopwords_ls)}] English stopwords imported from NLTK')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True


The first ten stopwords:


['himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don



There are [179] English stopwords imported from NLTK


In [39]:
# Define list of stopwords listed above that we should keep

stopword_keep_ls = []

"""
stopword_keep_ls = [
    'against',
    'not',
    'no',
    'nor',

    'do',
    'are',
    'is',
    'are',
    "not",
    'could',
    'did',
    'have',
    'has',
    'had',
    'might',
    'must',
    'need',
    'shall',
    'should',
    'was',
    'were',
    'would',

    'own',
    'same',
    'how',
    'why'
]
"""

'\nstopword_keep_ls = [\n    \'against\',\n    \'not\',\n    \'no\',\n    \'nor\',\n\n    \'do\',\n    \'are\',\n    \'is\',\n    \'are\',\n    "not",\n    \'could\',\n    \'did\',\n    \'have\',\n    \'has\',\n    \'had\',\n    \'might\',\n    \'must\',\n    \'need\',\n    \'shall\',\n    \'should\',\n    \'was\',\n    \'were\',\n    \'would\',\n\n    \'own\',\n    \'same\',\n    \'how\',\n    \'why\'\n]\n'

In [40]:
# Derive final stopword list

stopword_keep_ls = list(set(stopwords_ls) - set(stopword_keep_ls))
stopword_keep_ls
print('\n\n')
print(f'BEFORE: {len(stopwords_ls)} stopwords')
print(f' AFTER: {len(stopword_keep_ls)}')

['there',
 'aren',
 "mightn't",
 'an',
 'ma',
 'be',
 'she',
 "you've",
 'needn',
 'under',
 'was',
 'do',
 "aren't",
 'any',
 'themselves',
 'doesn',
 'we',
 'herself',
 'mightn',
 "wasn't",
 'too',
 "weren't",
 'been',
 'does',
 'to',
 'll',
 'should',
 'into',
 'again',
 'few',
 'over',
 'not',
 'above',
 'as',
 'nor',
 't',
 'have',
 'if',
 'you',
 'of',
 'further',
 "didn't",
 'most',
 'these',
 'with',
 'y',
 'the',
 'that',
 'has',
 'than',
 'yourselves',
 'did',
 'his',
 'm',
 's',
 'no',
 'haven',
 'mustn',
 "won't",
 'while',
 'can',
 'shan',
 'this',
 'which',
 "she's",
 "you'd",
 'ain',
 'am',
 'whom',
 'are',
 'a',
 'd',
 'it',
 'both',
 'shouldn',
 "shouldn't",
 'hers',
 'doing',
 'those',
 'other',
 'just',
 'own',
 'about',
 'because',
 'my',
 've',
 'himself',
 'very',
 "needn't",
 'below',
 'he',
 'up',
 "hasn't",
 "haven't",
 'now',
 "that'll",
 'for',
 'their',
 'each',
 'didn',
 'so',
 'weren',
 'don',
 'o',
 're',
 'yours',
 'how',
 'through',
 'i',
 'from',
 "don




BEFORE: 179 stopwords
 AFTER: 179


In [41]:
# Remove all stopwords with contractions (optional - could also remove fragments like 'll', 're', and 've' but not necessary)

stopwords_ls = [x for x in stopword_keep_ls if not "'" in x]
stopwords_ls[20:]
print(f'\n\nThere are now {len(stopwords_ls)} stopwords after removing words with contractions')


['ll',
 'should',
 'into',
 'again',
 'few',
 'over',
 'not',
 'above',
 'as',
 'nor',
 't',
 'have',
 'if',
 'you',
 'of',
 'further',
 'most',
 'these',
 'with',
 'y',
 'the',
 'that',
 'has',
 'than',
 'yourselves',
 'did',
 'his',
 'm',
 's',
 'no',
 'haven',
 'mustn',
 'while',
 'can',
 'shan',
 'this',
 'which',
 'ain',
 'am',
 'whom',
 'are',
 'a',
 'd',
 'it',
 'both',
 'shouldn',
 'hers',
 'doing',
 'those',
 'other',
 'just',
 'own',
 'about',
 'because',
 'my',
 've',
 'himself',
 'very',
 'below',
 'he',
 'up',
 'now',
 'for',
 'their',
 'each',
 'didn',
 'so',
 'weren',
 'don',
 'o',
 're',
 'yours',
 'how',
 'through',
 'i',
 'from',
 'more',
 'at',
 'between',
 'couldn',
 'is',
 'until',
 'myself',
 'but',
 'wasn',
 'or',
 'ourselves',
 'being',
 'such',
 'here',
 'him',
 'hasn',
 'me',
 'by',
 'on',
 'hadn',
 'theirs',
 'our',
 'itself',
 'in',
 'had',
 'after',
 'some',
 'only',
 'having',
 'won',
 'when',
 'were',
 'then',
 'isn',
 'who',
 'will',
 'down',
 'wouldn',




There are now 153 stopwords after removing words with contractions


**[CUSTOMIZE] Stopwords list by adding or deleting tokens**

In [42]:
# [CUSTOMIZE] Stopwords to ADD or DELETE from default NLTK English stopword list

STOPWORDS_ADD_SET = set(['bazinga', 'woo-hoo']) # Edit this list to add new words to the stopwords list
STOPWORDS_DEL_SET = set(['the']) # Edit this list to remove exising words from the stopwords list

stopwords_en_ls = list(set(stopwords_ls).difference(set(STOPWORDS_DEL_SET)).union(set(STOPWORDS_ADD_SET)))
print(f'Final Count after Customized Add/Del: {len(stopwords_en_ls)} Stopwords')

Final Count after Customized Add/Del: 154 Stopwords


#### Lemmatize

We use the WordNet lemmatizer from NLTK. A lemmatizer is preferred over a
stemmer in this case because it produces more readable words. Output that is
easy to read is very desirable in topic modelling.




In [43]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [44]:
%%time

# NOTE: 0m24s @03:48 on 20220228 Colab Pro 

# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

CPU times: user 4.82 s, sys: 82.2 ms, total: 4.9 s
Wall time: 4.93 s


We find bigrams in the documents. Bigrams are sets of two adjacent words.
Using bigrams we can get phrases like "machine_learning" in our output
(spaces are replaced with underscores); without bigrams we would only get
"machine" and "learning".

Note that in the code below, we find bigrams and then add them to the
original data, because we would like to keep the words "machine" and
"learning" as well as the bigram "machine_learning".

.. Important::
    Computing n-grams of large dataset can be very computationally
    and memory intensive.




### Identify Bi- and Tri-Grams

In [45]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

We remove rare words and common words based on their *document frequency*.
Below we remove words that appear in less than 20 documents or in more than
50% of the documents. Consider trying to remove words only based on their
frequency, or maybe combining that with this approach.




In [46]:
#@markdown Minimum number of documents a word must appear in: (default 20)

Min_No_Documents = 5 #@param {type:"slider", min:5, max:100, step:1}

#@markdown Max percent of documents a word can appear in: (default 0.50 or 50%)

Max_Percent_Documents = 0.49 #@param {type:"slider", min:0.05, max:0.9, step:0.01}


In [47]:
# docs_ls_ls = [astring.split() for astring in docs]
# docs = docs_ls_ls

In [48]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=Min_No_Documents, no_above=Max_Percent_Documents)

Finally, we transform the documents to a vectorized form. We simply compute
the frequency of each word, including the bigrams.




In [49]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

Let's see how many tokens and documents we have to train on.




In [50]:
# Corpus as a list of documents, each a list of tokens identifyed by dictionary tuples

len(corpus)

2999

In [51]:
type(corpus[0])

list

In [52]:
corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 3),
 (9, 1)]

In [53]:
print('Number of unique tokens: %d' % len(dictionary))  # Orig 1864
print('Number of documents: %d' % len(corpus))          # Orig 1740

Number of unique tokens: 9773
Number of documents: 2999


## Training

We are ready to train the LDA model. We will first discuss how to set some of
the training parameters.

First of all, the elephant in the room: how many topics do I need? There is
really no easy answer for this, it will depend on both your data and your
application. I have used 10 topics here because I wanted to have a few topics
that I could interpret and "label", and because that turned out to give me
reasonably good results. You might not need to interpret all your topics, so
you could use a large number of topics, for example 100.

``chunksize`` controls how many documents are processed at a time in the
training algorithm. Increasing chunksize will speed up training, at least as
long as the chunk of documents easily fit into memory. I've set ``chunksize =
2000``, which is more than the amount of documents, so I process all the
data in one go. Chunksize can however influence the quality of the model, as
discussed in Hoffman and co-authors [2], but the difference was not
substantial in this case.

``passes`` controls how often we train the model on the entire corpus.
Another word for passes might be "epochs". ``iterations`` is somewhat
technical, but essentially it controls how often we repeat a particular loop
over each document. It is important to set the number of "passes" and
"iterations" high enough.

I suggest the following way to choose iterations and passes. First, enable
logging (as described in many Gensim tutorials), and set ``eval_every = 1``
in ``LdaModel``. When training the model look for a line in the log that
looks something like this::

   2016-06-21 15:40:06,753 - gensim.models.ldamodel - DEBUG - 68/1566 documents converged within 400 iterations

If you set ``passes = 20`` you will see this line 20 times. Make sure that by
the final passes, most of the documents have converged. So you want to choose
both passes and iterations to be high enough for this to happen.

We set ``alpha = 'auto'`` and ``eta = 'auto'``. Again this is somewhat
technical, but essentially we are automatically learning two parameters in
the model that we usually would have to specify explicitly.




In [54]:
#@markdown How many Topics do you want to find?

No_of_Topics = 10 #@param {type:"slider", min:2, max:200, step:1}

#@markdown Default 10-50 depending upon how large the text and diverse the vocabulary

In [55]:
%%time

# NOTE: 2m01s @20220328 with Google Colab/CPU on Harry Potter

# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = No_of_Topics
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

CPU times: user 1min 9s, sys: 707 ms, total: 1min 10s
Wall time: 1min 11s


We can compute the topic coherence of each topic. Below we display the
average topic coherence and print the topics in order of topic coherence.

Note that we use the "Umass" topic coherence measure here (see
:py:func:`gensim.models.ldamodel.LdaModel.top_topics`), Gensim has recently
obtained an implementation of the "AKSW" topic coherence measure (see
accompanying blog post, http://rare-technologies.com/what-is-topic-coherence/).

If you are familiar with the subject of the articles in this dataset, you can
see that the topics below make a lot of sense. However, they are not without
flaws. We can see that there is substantial overlap between some topics,
others are hard to interpret, and most of them have at least some terms that
seem out of place. If you were able to do better, feel free to share your
methods on the blog at http://rare-technologies.com/lda-training-tips/ !




In [56]:
%whos str

Variable            Type    Data/Info
-------------------------------------
TEXT_ENCODING       str     utf-8
Text_Title          str     [pashinyan] from [20211223] to [20221214]
adoc                str     The Armenian police confi<...>ş https://t.co/cP3q3RSfNu
aline               str     Pashinyan's government an<...>ship of Pashinyan!\n\n6/6
doc_temp_str        str     @Gegham_Artsakh If Pashin<...>ivil Contract from power.
end_date            str     20221214
end_time            str     234645.csv
fn                  str     tweets_raw_pashinyan_2021<...>45418_20221214_234645.csv
search_str          str     pashinyan
start_date          str     20211223
start_time          str     145418
temp                str     against
text_filename_str   str     tweets_raw_pashinyan_2021<...>45418_20221214_234645.csv
text_raw_str        str     #armenianbreakingnews\nTo<...>ivil Contract from power.
text_title_str      str     Pashinyan From 20211223 To 20221214
token               str     po

In [57]:
search_str

'pashinyan'

In [58]:
top_topics = model.top_topics(corpus) #, num_words=20)


# from pprint import pprint
# pprint(top_topics)

# print('\n\n')
# for i,atopic in enumerate(top_topics):
#   atopic_coherence_str = f'Topic #{i}: coherence = {atopic[1]}'
#   print(atopic_coherence_str)
#   report_line_ls.append(atopic_coherence_str)


topic_word_ls = []

topic_word_ls.append(text_title_str)

# print(text_title_str)

for i, atopic in enumerate(top_topics):
  topic_str = f'\n\nTopic #{i}: {atopic[1]} (coherence)'
  # print(topic_str)
  topic_word_ls.append(topic_str)
  for j, aword in enumerate(atopic[0]):
    word_str = f'  Word #{j}: {aword}'
    # print(word_str)
    topic_word_ls.append(word_str)

avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
avg_topic_coherence_str = f'Average topic coherence: {avg_topic_coherence:.4f}'
# print(avg_topic_coherence_str)
topic_word_ls.append(f'\n\n{avg_topic_coherence_str}')

topic_word_str = "\n".join(topic_word_ls)
print(topic_word_str)

# Save to file
filename_topic_word_str = f'lda_topic_{search_str}_word_report.txt'
with open(filename_topic_word_str, 'w') as fp:
  fp.write(topic_word_str)

files.download(filename_topic_word_str)

Pashinyan From 20211223 To 20221214


Topic #0: -0.9798185714125117 (coherence)
  Word #0: (0.008109091, 'they')
  Word #1: (0.0072152447, 'we')
  Word #2: (0.006292118, 'what')
  Word #3: (0.006114858, 'who')
  Word #4: (0.006007694, 'artsakh')
  Word #5: (0.0056505674, 'do')
  Word #6: (0.0056440304, 'if')
  Word #7: (0.0055348203, 'amp')
  Word #8: (0.0055285776, 'all')
  Word #9: (0.005451664, 'russia')
  Word #10: (0.005254061, 'war')
  Word #11: (0.0051376545, 'people')
  Word #12: (0.004872185, 'so')
  Word #13: (0.004548986, 'or')
  Word #14: (0.004539605, 'nikolpashinyan')
  Word #15: (0.0045380443, 'like')
  Word #16: (0.0044743367, 'can')
  Word #17: (0.0044420063, 'about')
  Word #18: (0.004301543, 'your')
  Word #19: (0.004269742, 'our')


Topic #1: -1.1019840107782222 (coherence)
  Word #0: (0.027230132, 'el')
  Word #1: (0.015590216, 'en')
  Word #2: (0.012154254, 'aliyev')
  Word #3: (0.010918953, 'ministro')
  Word #4: (0.0104193855, 'primer')
  Word #5: (0.01009637, '

7683

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Visualize

In [59]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [60]:
# vis_data = gensimvis.prepare(lda, corpus, dictionary)
vis_data = gensimvis.prepare(model, corpus, dictionary)
pyLDAvis.display(vis_data)

## Things to experiment with

* ``no_above`` and ``no_below`` parameters in ``filter_extremes`` method.
* Adding trigrams or even higher order n-grams.
* Consider whether using a hold-out set or cross-validation is the way to go for you.
* Try other datasets.

## Where to go from here

* Check out a RaRe blog post on the AKSW topic coherence measure (http://rare-technologies.com/what-is-topic-coherence/).
* pyLDAvis (https://pyldavis.readthedocs.io/en/latest/index.html).
* Read some more Gensim tutorials (https://github.com/RaRe-Technologies/gensim/blob/develop/tutorials.md#tutorials).
* If you haven't already, read [1] and [2] (see references).

## References

1. "Latent Dirichlet Allocation", Blei et al. 2003.
2. "Online Learning for Latent Dirichlet Allocation", Hoffman et al. 2010.


